In [3]:
import numpy as np
import pandas as pd
import hyperopt
from catboost import Pool, CatBoostRegressor, cv, CatboostIpythonWidget
#Note does not include latest batch of features

/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
train_2016 = pd.read_csv('../datasets/train_2016_v2.csv', parse_dates=['transactiondate'])
train_2017 = pd.read_csv('../datasets/train_2017.csv', parse_dates=['transactiondate'])
properties_2016 = pd.read_csv('../datasets/properties_2016.csv')
properties_2017 = pd.read_csv('../datasets/properties_2017.csv')

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = df["transactiondate"].dt.month
    df["transaction_quarter"] = df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

In [6]:
train_2016 = add_date_features(train_2016)
train_2017 = add_date_features(train_2017)
train_2016 = train_2016.merge(properties_2016, how='left', on='parcelid')
train_2017 = train_2017.merge(properties_2017, how='left', on='parcelid')
train_df = train_2016.append(train_2017)

In [7]:
missing_perc_thresh = 0.98
exclude_missing = []
num_rows = train_df.shape[0]
for c in train_df.columns:
    num_missing = train_df[c].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(num_rows)
    if missing_frac > missing_perc_thresh:
        exclude_missing.append(c)

In [8]:
exclude_unique = []
for c in train_df.columns:
    num_uniques = len(train_df[c].unique())
    if train_df[c].isnull().sum() != 0:
        num_uniques -= 1
    if num_uniques == 1:
        exclude_unique.append(c)

In [9]:
exclude_other = ['parcelid', 'logerror']  # for indexing/training only
# do not know what this is LARS, 'SHCG' 'COR2YY' 'LNR2RPD-R3' ?!?
exclude_other.append('propertyzoningdesc')
train_features = []
for c in train_df.columns:
    if c not in exclude_missing \
       and c not in exclude_other and c not in exclude_unique:
        train_features.append(c)

In [10]:
cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(train_features):
    num_uniques = len(train_df[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)

In [11]:
train_df.fillna(-999, inplace=True)

In [18]:
def hyperopt_objective(params):

    model = CatBoostRegressor(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        bagging_temperature=params['bagging_temperature'],
        rsm=params['rsm'],
        depth=params['depth'],
        border_count=params['border_count'],
        ctr_border_count=params['ctr_border_count'],
        iterations=200,
        eval_metric='MAE',
        loss_function='MAE'
    )
    cv_data = cv(
        model.get_params(),
        Pool(train_df[train_features], train_df['logerror'], cat_features=cat_feature_inds)
    )
    return np.min(cv_data['b\'MAE\'_test_avg'])

In [24]:

from itertools import combinations

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
    'bagging_temperature': hyperopt.hp.uniform('bagging_temperature', 0, 1),
    'depth': hyperopt.hp.choice('depth', np.arange(1, 11, dtype=int)),
    'rsm': hyperopt.hp.uniform('rsm', 0, 1),
    'border_count': hyperopt.hp.choice('border_count', np.arange(1, 256, dtype=int)),
    'ctr_border_count': hyperopt.hp.choice('ctr_border_count', np.arange(1, 256, dtype=int))
}
best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=55
)

print(best)

/anaconda/lib/python3.6/site-packages/catboost/core.py:1304: UserWarning: Parameter "use_best_model" has no effect in cross-validation and is ignored
  warnings.warn('Parameter "use_best_model" has no effect in cross-validation and is ignored')


{'bagging_temperature': 0.7218369037954099, 'border_count': 246, 'ctr_border_count': 197, 'depth': 7, 'l2_leaf_reg': 4.0, 'learning_rate': 0.030747575306233767, 'rsm': 0.8506573951709594}


In [ ]:
#training on entire ensemble impractical, much too long to run anything
%%timeit
num_ensembles=5
accuracy=0.0
for i in range(num_ensembles):
    # TODO(you): Use CV, tune hyperparameters
    print('Training ' + str(i))
    model = CatBoostRegressor(
        iterations=200,
        eval_metric='MAE',
        loss_function='MAE',
        random_seed=1
    )
    cv_data = cv(
        model.get_params(),
        Pool(train_df[train_features], train_df['logerror'], cat_features=cat_feature_inds)
    )
    accuracy += np.min(cv_data['b\'MAE\'_test_avg'])

Training 0


/anaconda/lib/python3.6/site-packages/catboost/core.py:1304: UserWarning: Parameter "use_best_model" has no effect in cross-validation and is ignored
  warnings.warn('Parameter "use_best_model" has no effect in cross-validation and is ignored')


Training 1
